# Java中间件的定义

    In its most general sense, middleware is computer software that provides services to software applications beyond those available from the operating system. Middleware can be described as "software glue". Thus middleware is not obviously part of an operating system, not a database management system, and neither is it part of one software application. Middleware makes it easier for software developers to perform communication and input/output, so they can focus on the specific purpose of their application.
中间件分类:

1. 远程过程调用和对象访问中间件:主要解决分布式环境下应用的互相访问问题,支撑应用服务化的基础.Eureka/Dubbo
2. 消息中间件:解决应用之间的消息传递/解耦/异步的问题,redis/rabbitmq
3. 数据访问中间件:主要解决应用访问数据库的共性问题的组件

# 构建Java中间件的基础知识

## JVM
    Java虚拟机识别的是统一格式的中间代码 - Java Byte Code.
## 垃圾回收与内存堆布局
    Java虚拟机是通过垃圾回收的方式来进行内存回收的.
    以Oracle Hotspot JVM中内存的堆布局为例, 包括Young/Tenured/Perm三块区域,即新生代/老年代/持久代.
    一般来说,新的对象会被分配在新生代(Young)的Eden区,也有可能会被直接分配在老年代(Tenured).在进行新生代垃圾回收的时候,Eden区中存活的对象会被复制到空的Survivor区,而下次新生代垃圾回收的时候,Eden区存活的对象和这个Survivor区中存活的对象会被复制到另外那个Survivor区,并且清空当前Survivor区.经过多次新生代垃圾回收,还存活的对象会被移动到年老代.
常见的GC算法:
1. 针对新生代
    1. 串行GC - Serial Copying
    2. 并行GC - ParNew
    3. 并行回收GC - Parallel Scavenge
2. 针对老年代
    1. 串行GC - Serial MSC
    2. 并行MS GC - Parallel MSC
    3. 并发Compacting GC - Parallel Compacting
    4. 并发GC - CMS
3. Java1.6引入了G1(Garbage First)回收器,以取代CMS
## Java并发编程的类/接口和方法
### 线程池
    与每次需要时都创建线程相比,线程池可以降低创建线程的开销,因为线程池在线程执行结束后进行的是回收操作,而不是真正销毁线程.
### synchronized
    synchronized可以用于声明方法,也可以用于声明代码块.推荐用synchronized修饰代码块,因为更灵活,可以传参,且可以是任意对象.
### ReenrantLock - 可重入锁
    ReenrantLock的用法类似修饰代码段的synchronized,不过需要显式进行unlock.对比synchronized的好处:
    1. ReentrantLock提供了tryLock方法,tryLock调用时,如果锁被其它线程持有,那么tryLock立即返回,返回结果为false;如果锁没有被其它线程持有,则当前线程持有锁,返回结果为true.
    2. 构造ReentrantLock对象时,有一个构造函数可以接受一个boolean类型的参数,就是描述锁公平与否的函数.公平锁的好处是等待的线程不会饿死,但整体效率相对较低;非公平锁的好处是整体效率较高,但有些线程可能会饿死或很早就在等待锁,但要等很久才能得到锁.原因是,公平锁是严格按照请求锁的顺序来排队获取锁,而非公平锁是可以抢占的.
    3. ReentrantLock提供了ReentrantReadWriteLock,即读写锁,主要用与读多写少并且读不需要互斥的场景,这种场景使用读写锁会比使用全部互斥的锁的性能高很多.
    注: ReentrantReadWriteLock与ReentrantLock的用法类似,差异是ReentrantReadWriteLock通过readLock()和writeLock()来获得相关的读锁和写锁操作.
### volatile
Java并发特性
1. 原子性是指一个操作是不可中断的。即使是在多个线程一起执行的时候，一个操作一旦开始，就不会被其它线程干扰
2. 有序性即程序执行的顺序按照代码的先后顺序执行
3. 可见性是指当多个线程访问同一个变量时，一个线程修改了这个变量的值，其他线程能够立即看得到修改的值
volatile关键字是轻量级的实现变量可见性的方法,因为volatile只是保证同一个变量在多线程中的可见性,所以更多的用于修饰作为开关状态的变量.注意:volatile虽然解决了可见性的问题,但不能控制并发.
### Atomics
    JDK5增加了java.util.concurrent.atomic包,提供了一些以Atomic开头的类,这些类提供了一些相关的原子操作.包括各基本数据类型和数组的原子类,例如AtomicBoolean/AtomicInteger/AtomicLong/AtomicReference/AtomicIntegerArray/AtomicLongArray/AtomicReferenceArray ...
### wait/notify和notifyAll
    wait/notify/notifyAll是Java的Object对象的三个方法,在多线程中,可以把某个对象作为事件对象,通过这个对象的wait/notify/notifyAll方法来完成线程间的状态通知.
    在实践中,对wait的使用一般是嵌在一个循环中,且会判断相关的数据状态是否达到预期,如果没有则会继续等待,主要为了防止虚假唤醒.
### CountDownLatch
CountDownLatch是java.util.concurrent包中的一个类.主要提供的机制是当多个(具体数量等于初始化CountDownLatch时的count参数的值)线程都到达了预期状态或完成预期工作时触发事件,其它线程可以等待这个事件来触发自己后续的工作.需注意的几点:
1. 等待的线程可以是多个,即CountDownLatch是可以唤醒多个等待的线程的
2. 到达自己预期状态的线程会调用CountDownLatch的countDown方法
3. 等待的线程会调用CountDownLatch的await方法
### CyclicBarrier
CyclicBarrier,字面意思循环屏障.CyclicBarrier可以协调多个线程,让多个线程在这个屏障前等待,直到所有线程都到达了这个屏障时,再一起执行后面的动作.
<b>
CyclicBarrier和CountDownLatch都是用于多个线程间的协调的.二者的一个很大的区别是,CountDownLatch是在多个线程都进行了latch.countDown后才会触发事件,唤醒await在latch上的线程,而执行countDown的线程,执行完countDown后会继续自己线程的工作;CyclicBarrier是一个栅栏,用于同步所有调用await方法的线程,并且等所有线程都到了await方法时,这些线程才一起返回各自的工作(因为使用CyclicBarrier的线程都会阻塞在await方法上,所以线程池中使用CyclicBarrier要特别小心,如果线程池的线程数过少,就会发生死锁).此外,CountDownLatch与CyclicBarrier还有一个差别,那就是CountDownLatch不能循环使用,CyclicBarrier可以循环使用.
</b>

### Future和FutureTask
Future是一个接口,FutureTask是一个具体实现类.配合Callable接口使用,或者ThreadPoolExecutor的submit/submitAll方法,返回Future/Future集合
### 并发容器
并发容器时线程安全容器的一种,但强调容器的并发性,即不仅追求线程安全,还要考虑并发性,提升在容器并发环境下的性能.不同于加锁互斥的方式完成线程安全,并发容器的思路时尽量不用锁,比较代表性的是以CopyOnWrite和Concurrent开头的几个容器.
<b>
CopyOnWrite的思路是在更改容器的时候,把容器写一份进行修改,保证正在读的线程不受影响,这种方式用在读多写少的场景中会非常好,因为实质上是在写的时候重建了一个容器.而以Concurrent开头的容器的具体实现方式则不完全相同,总体来说是尽量保证读不加锁,并且修改不影响读,所以会达到比使用读写锁更高的并发性能.eg:CopyOnWriteArrayList/CopyOnWriteArraySet/ConcurrentHashMap/ConcurrentSkipListMap/ConcurrentLinkedQueue/ConcurrentLinkedDeque ...
</b>    

## 动态代理
   静态代理 - 实现简单,存在问题: 如果需要对多个类进行代理,且代理类的功能实现是一致的,就需要对每一个具体的类都完成一个代理类,造成重复代码.
   
<code>
    /**
     * 静态代理
     */
    public class CalculatorImpl implements Calculator {
        public int add(int a, int b) {
            return a + b;
        }
    }
    public class CalculatorProxy implements Calculator {
        private Calculator calculator;
        CalculatorProxy (Calculator calculator) {
            this.calculator = calculator;
        }
        public int add(int a, int b) {
            // 具体执行前可做的工作
            int result = calculator.add(a,b);
            // 具体执行后可做的工作
            return result;
        }
    }
</code>
    动态代理 - 动态地生成具体委托类的代理类实现对象.与静态代理不同,动态代理并不需要为各个委托类逐一实现代理类,只需要为一类代理行为写一个具体的实现类即可.通过Proxy.newProxyInstance来创建代理的方法可以为不同的委托类都创建代理类.主要在invoke方法内部完成方法增强.
    
<code>
    /**
     * 动态代理
     */
    public void testDynamicProxy() {
        Calculator calculator = new CalculatorImple();
        LogHandler lh = new LogHandler(calculator);
        Calculator proxy = (Calculator) Proxy.newProxyInstance(calculator.getClass().getClassLoader(),
                                                            calculator.getClass().getInterfaces(),lh);
        proxy.add(1,1);
    }
    public class LogHandler implements InvocationHandler {
        Object obj;
        LogHandler (Object obj) {
            this.obj = obj;
        }
        public Object invoke(Object obj1,Method method,Object[] args) throws Throwable {
            this.doBefore();
            Object o = method.invoke(obj,args);
            this.doAfter();
            return o;
        }
        public void doBefore(){
            System.out.println("Do something before");
        }
        public void doAfter(){
            System.out.println("Do something after");
        }
    }
</code>

## 反射
Java反射机制是指在运行状态中,对于任何一个类,都能够知道这个类的所有属性和方法;对于任何一个对象,都能够调用它的任何一个方法和属性.Java反射机制主要提供了以下功能:
1. 在运行时判断任意一个对象所属的类
2. 在运行时构造任何一个类的对象
3. 在运行时判断任何一个类所具有的成员变量和方法
4. 在运行时调用任何一个对象的方法
5. 生成动态代理
Java的反射机制为Java本身带来了动态性,能够让代码更灵活,用法示例:
1. 获取对象属于哪个类
    Class clazz = object.getClass();
2. 获取类的信息
    String className = clazz.getName();
    Method[] methods = clazz.getDeclaredMethods();
    Field[] fields = clazz.getDeclaredFields();
3. 构建对象
    Class.forName("ClassName").newInstance();
4. 动态执行方法
    Method method = clazz.getDeclaredMethod("add",int.class,int.class);
    method.invoke(this,1,1);
5. 动态操作属性
    Field field = clazz.getDeclaredField("name");
    field.set(this,"Test");
除了Java自身提供的动态代理和反射的支持外,字节码增强也是经常会用到的技术,并且有第三方库可以直接使用,如cglib,asm...

# 分布式系统中的Java中间件

    分布式系统的Java中间件是指 用来支撑网站从小到大的变化过程中解决应用拆分/服务拆分/数据拆分和应用解耦问题的产品.
    服务框架帮助对应用进行拆分,完成服务化;
    数据层帮助完成数据的拆分以及整个数据的管理/扩容/迁移等工作;
    消息中间件帮助完成应用的解耦,并提供一种分布式环境下完成事务的思路.